In [126]:
import pandas as pd
import numpy as np
import folium


from statistics import mean 
from IPython.display import Markdown
from opencage.geocoder import OpenCageGeocode
from ipyleaflet import *
from ipywidgets import HTML


def special_print(title, content):
    display(Markdown("### %s:" % title))
    print(content)
    return None
    
    
class getBiodiversity():

    def __init__(self, url):
        self.url = url
        try:
            self.df_data = pd.read_csv(url, sep=';', header=0, encoding='utf-8')
        except Exception as e:
            self.df_data = pd.DataFrame()
            print("Aborting... couldn't read this file: %s" % url)
            print (e.args)
        self.data_info = "File shape: %d rows x %d columns"% (self.df_data.shape[0], self.df_data.shape[1])
        return None
    
    def getColumns(self):
        self.df_columns = list(self.df_data.columns)
        return None
    
    def checkEmpty(self):
        self.getColumns()
        self.df_dataNAN = pd.DataFrame(np.where((self.df_data == '') | (self.df_data == 'Sem Informações'), 1, 0))
        self.df_dataNAN.columns = self.df_columns
        self.df_data_missing = 100*self.df_dataNAN.mean()
        return None

    def getLastFilled(self, columns):
        filled_columns = [column for column in columns if (column != "Sem Informações")]
        return 'NA' if len(filled_columns) == 0 else TAXONOMY_COLUMNS[len(filled_columns)-1]
    
    def addTaxonomicLevel(self, col_name):
        self.df_data[col_name] = self.df_data[TAXONOMY_COLUMNS].apply(lambda x: self.getLastFilled(x), axis=1)
        self.df_taxonomy_info =  self.df_data[col_name].value_counts()
        return None

    def extractTaxonomy(self, columns):
        self.df_taxonomy = self.df_data[columns]
        return None
    
    def getTaxonomy(self, col_name='taxonomic_level'):
        self.addTaxonomicLevel(col_name)
        self.extractTaxonomy(TAXONOMY_COLUMNS+[col_name])
        return None
    
    def filterFields(self, columns, values):
        filter = np.logical_and.reduce([self.df_data[columns[i]].isin(values[i]) for i in range(len(columns))])
        self.df_filtered = self.df_data[filter]
        self.filtered_info = "File shape: %d rows x %d columns"% (self.df_filtered.shape[0], self.df_filtered.shape[1])
        return None
    
    def parseFloat(self, info):
        value = float(info)
        try:
            value = float(info)
        except:
            value = 0.0
        return value
    
    def checkGeoInfo(self, components, reported):
        aux = []
        unmatched = 0
        for elem in ["country", "state", "state_code", "city"]:
            try:
                value = components[elem]
            except:
                value = "NA"
            aux.append(value)
        unmatched += 1 if reported[0] != aux[0] else 0
        unmatched += 1 if not reported[1] in [aux[1], aux[2]] else 0
        unmatched += 1 if reported[2] != aux[3] else 0
        return unmatched
    
    def reverseGeocode(self, latlon):
        geo = geocoder.reverse_geocode(latlon[0], latlon[1], no_annotations = '1', pretty = '1', language='pt')
        comp = geo[0]['components']
        info = self.checkGeoInfo(comp, [latlon[2], latlon[3], latlon[4]])
        return pd.Series((geo[0]['formatted'], info))
    
    def printMap(self):
        coords = self.df_location_sample[["AdjustedLatitude", "AdjustedLongitude", "ReversedAddress", "Confidence"]].T.values.tolist()
        COLORS = ['green', 'lightgreen', 'orange', 'red']
        center = [mean(coords[0][:]), mean(coords[1][:])]
        my_map = folium.Map(location=center, zoom_start=3, tiles="OpenStreetMap")
        for i in range(len(self.df_location_sample)):
            folium.Marker(location=[coords[0][i], coords[1][i]], popup=coords[2][i], 
                          icon=folium.Icon(color=COLORS[coords[3][i]], icon='map-marker')).add_to(my_map) 
        self.observations_map = my_map
        return None
        
    def checkCoordinates(self, size):
        self.df_data["AdjustedLatitude"] = self.df_data["Latitude"].apply(lambda x: self.parseFloat(x))
        self.df_data["AdjustedLongitude"] = self.df_data["Longitude"].apply(lambda x: self.parseFloat(x))
        self.df_location_sample = self.df_data.sample(n=size)
        self.df_location_sample[['ReversedAddress','Confidence']] = self.df_location_sample[['AdjustedLatitude','AdjustedLongitude']+LOCATION_COORDINATES].apply(self.reverseGeocode, axis=1)
        self.printMap()
        return None



####################################################################################
#
# Initializing all data

url = "portalbio_export_16-10-2019-14-39-54.csv"
TAXONOMY_COLUMNS = ['Filo', 'Classe', 'Ordem', 'Familia', 'Genero', 'Especie']
LOCATION_COORDINATES = ['Pais', 'Estado/Provincia', 'Municipio', 'Latitude', 'Longitude']
LOCATION_SAMPLING = 20 # number of locations to check

key = '09aadb1b1d8840acacfa0fcece0acb13'
geocoder = OpenCageGeocode(key)

biodiversity = getBiodiversity(url)
biodiversity.checkEmpty()
biodiversity.getTaxonomy(col_name='Nível Taxonômico')
biodiversity.filterFields(['Municipio','Filo'],[['Fernando De Noronha','Niquelândia'],['Mollusca','Annelida']])
biodiversity.checkCoordinates(LOCATION_SAMPLING)

In [127]:
####################################################################################
#
# Show sample of each output - show map with reported observations

display(Markdown("### Observations (click to see more details)"))

### Observations (click to see more details)

In [128]:
biodiversity.observations_map

In [4]:
####################################################################################
#
# Show sample of each output - raw data load

special_print("File URL", biodiversity.url)
special_print("Raw file info", biodiversity.data_info)
special_print("Raw file sample", biodiversity.df_data.head(1).T)
special_print("Dataframe columns", biodiversity.df_columns)

### File URL:

portalbio_export_16-10-2019-14-39-54.csv


### Raw file info:

File shape: 3912 rows x 34 columns


### Raw file sample:

                                                                                     0
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                            Atlas de Registro de Aves Brasileiras
Sigla da base de dados                    ARA - Atlas de Registros de Aves Brasileiras
Responsavel pelo registro            Ibama (Instituto Brasileiro de Meio Ambiente e...
Numero do registro no portal                                             ARA-LIT-95641
Numero do registro na base de dados                                    Sem Informações
Data do registro                                                       Sem Informações
Data do evento                                                         Sem Informações
Data de Carencia                                                            27/09/2017
Nome cientifico                            

### Dataframe columns:

['Nome da instituicao', 'Sigla da instituicao', 'Nome da base de dados', 'Sigla da base de dados', 'Responsavel pelo registro', 'Numero do registro no portal', 'Numero do registro na base de dados', 'Data do registro', 'Data do evento', 'Data de Carencia', 'Nome cientifico', 'Nome comum', 'Nome cientifico na base de dados', 'Nivel taxonomico', 'Numero de individuos', 'Reino', 'Filo', 'Classe', 'Ordem', 'Familia', 'Genero', 'Especie', 'Estado de conservacao', 'Categoria de Ameaca', 'Localidade', 'Pais', 'Estado/Provincia', 'Municipio', 'Status de Sensibilidade', 'Latitude', 'Longitude', 'Outras informacoes da localidade', 'Jurisdicao', 'Destino do Material']


In [350]:
####################################################################################
#
# Show sample of each output - data missing analysis

special_print("Data missing sample (1 = missing)", biodiversity.df_dataNAN.head(5).T)
special_print("Data missing statistics (%)", biodiversity.df_data_missing)

### Data missing sample (1 = missing):

                                     0  1  2  3  4
Nome da instituicao                  1  1  1  1  1
Sigla da instituicao                 0  0  0  0  0
Nome da base de dados                0  0  0  0  0
Sigla da base de dados               0  0  0  0  0
Responsavel pelo registro            0  0  0  0  0
Numero do registro no portal         0  0  0  0  0
Numero do registro na base de dados  1  1  1  1  1
Data do registro                     1  1  1  1  1
Data do evento                       1  1  1  1  1
Data de Carencia                     0  0  0  0  0
Nome cientifico                      0  0  0  0  0
Nome comum                           0  0  0  0  0
Nome cientifico na base de dados     0  0  0  0  0
Nivel taxonomico                     0  0  0  0  0
Numero de individuos                 0  0  0  0  0
Reino                                1  1  1  1  1
Filo                                 0  0  0  0  0
Classe                               0  0  0  0  0
Ordem                          

### Data missing statistics (%):

Nome da instituicao                    100.000000
Sigla da instituicao                     0.000000
Nome da base de dados                    0.000000
Sigla da base de dados                   0.000000
Responsavel pelo registro                0.076687
Numero do registro no portal             0.000000
Numero do registro na base de dados      6.007157
Data do registro                         3.502045
Data do evento                          54.447853
Data de Carencia                         0.000000
Nome cientifico                          1.687117
Nome comum                              74.207566
Nome cientifico na base de dados         0.000000
Nivel taxonomico                         1.687117
Numero de individuos                    51.354806
Reino                                  100.000000
Filo                                     3.169734
Classe                                   3.757669
Ordem                                    4.294479
Familia                                  4.729039


In [351]:
####################################################################################
#
# Show sample of each output - show taxonomic info

special_print("Raw data sample after taxonomic level inclusion", biodiversity.df_data.head(1).T)
special_print("Taxonomic info", biodiversity.df_taxonomy_info)
special_print("Taxonomy sample", biodiversity.df_taxonomy.head(3).T)

### Raw data sample after taxonomic level inclusion:

                                                                                     0
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                            Atlas de Registro de Aves Brasileiras
Sigla da base de dados                    ARA - Atlas de Registros de Aves Brasileiras
Responsavel pelo registro            Ibama (Instituto Brasileiro de Meio Ambiente e...
Numero do registro no portal                                             ARA-LIT-95641
Numero do registro na base de dados                                    Sem Informações
Data do registro                                                       Sem Informações
Data do evento                                                         Sem Informações
Data de Carencia                                                            27/09/2017
Nome cientifico                            

### Taxonomic info:

Especie    2447
Genero      819
Familia     461
NA          124
Filo         23
Classe       21
Ordem        17
Name: Nível Taxonômico, dtype: int64


### Taxonomy sample:

                                   0              1              2
Filo                        Chordata       Chordata       Chordata
Classe                          Aves           Aves           Aves
Ordem                  Ciconiiformes  Ciconiiformes  Ciconiiformes
Familia                 Scolopacidae        Laridae        Laridae
Genero                       Actitis          Anous          Anous
Especie           Actitis macularius  Anous minutus  Anous minutus
Nível Taxonômico             Especie        Especie        Especie


In [352]:
####################################################################################
#
# Show sample of each output - filtered data

special_print("Filtered data info", biodiversity.filtered_info)
special_print("Filtered data sample", biodiversity.df_filtered.head(1).T)

### Filtered data info:

File shape: 208 rows x 35 columns


### Filtered data sample:

                                                                                  3129
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                Sistema de Autorização e Informação em Biodive...
Sigla da base de dados                                                    SISBIO-DIBIO
Responsavel pelo registro                                     Fábio Hissa Vieira Hazin
Numero do registro no portal                                                    527175
Numero do registro na base de dados  Nº Da Autorização/Licença Sisbio: 17081, Proje...
Data do registro                                                            03/05/2014
Data do evento                                                 04/05/2014 a 06/05/2014
Data de Carencia                                                            04/10/2019
Nome cientifico                            

In [353]:
####################################################################################
#
# Show sample of each output - show location info

special_print("Raw data sample after location adjustments", biodiversity.df_data.head(1).T)
special_print("Sample of locations to check", biodiversity.df_location_sample.head(1).T)

### Raw data sample after location adjustments:

                                                                                     0
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                            ICMBio
Nome da base de dados                            Atlas de Registro de Aves Brasileiras
Sigla da base de dados                    ARA - Atlas de Registros de Aves Brasileiras
Responsavel pelo registro            Ibama (Instituto Brasileiro de Meio Ambiente e...
Numero do registro no portal                                             ARA-LIT-95641
Numero do registro na base de dados                                    Sem Informações
Data do registro                                                       Sem Informações
Data do evento                                                         Sem Informações
Data de Carencia                                                            27/09/2017
Nome cientifico                            

### Sample of locations to check:

                                                                                    48
Nome da instituicao                                                    Sem Informações
Sigla da instituicao                                                              JBRJ
Nome da base de dados                    Herbário do Jardim Botânico do Rio de Janeiro
Sigla da base de dados                                                              RB
Responsavel pelo registro                                           Gª Pedra do Cavalo
Numero do registro no portal                                                RB00079609
Numero do registro na base de dados                                                953
Data do registro                                                       Sem Informações
Data do evento                                                         Sem Informações
Data de Carencia                                                            30/10/2017
Nome cientifico                            